In [1]:
import tensorflow as tf

/home/phunc20/.virtualenvs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/phunc20/.virtualenvs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/phunc20/.virtualenvs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/phunc20/.virtualenvs/homl1e/lib/python3.7/s

In [2]:
n_inputs = 3   # This is the k in recurrent-neurons.pdf
n_neurons = 5  # This is the l in recurrent-neurons.pdf

### RNN manually

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

#Wx = tf.Variable(tf.random_normal(shape=[k,l], dtype=tf.float32))
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
#Wy = tf.Variable(tf.random_normal(shape=[l,l], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
#b = tf.Variable(tf.zeros(shape=[1,l], dtype=tf.float32))
b = tf.Variable(tf.zeros(shape=[1, n_neurons], dtype=tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
Y0 = tf.tanh(X0 @ Wx + b)
#Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(X1 @ Wx + Y0 @ Wy + b)
#Y1 = tf.tanh(tf.matmul(X1, Wx) + tf.matmul(Y0, Wy) + b)

In [5]:
Y0, Y1

(<tf.Tensor 'Tanh:0' shape=(?, 5) dtype=float32>,
 <tf.Tensor 'Tanh_1:0' shape=(?, 5) dtype=float32>)

In [6]:
init = tf.global_variables_initializer()

In [7]:
import numpy as np
# batch size: 4
# mini-batch:        instance0, instance1, instance2, instance3
X0_batch = np.array([[0,1,2],   [3,4,5],   [6,7,8],   [9,0,1],])  # t = 0
X1_batch = np.array([[9,8,7],   [0,0,0],   [6,5,4],   [3,2,1],])  # t = 1

In [8]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [9]:
print(f"Y0_val =\n{Y0_val}")
print()
print(f"Y1_val =\n{Y1_val}")

Y0_val =
[[ 0.16864021 -0.9980501  -0.12999508  0.5956525   0.58439237]
 [-0.80643463 -0.99999803 -0.6358939   0.9998137   0.99734855]
 [-0.98379624 -1.         -0.87909365  0.9999999   0.99998665]
 [ 0.76022303  0.99999976  0.55756396  0.99981666  0.996303  ]]

Y1_val =
[[-0.99979436 -0.9999987  -0.36007643  1.          1.        ]
 [-0.8384599   0.49807075  0.99434614  0.9998125   0.99435616]
 [-0.99952406 -0.9930898   0.9662427   1.          0.9999995 ]
 [-0.97035104 -0.84122384 -0.8353216   0.99982643  0.9801899 ]]


The output numerical values like above have little meaning in this notebook, so don't pay too much attention to them.

### `static_rnn()`
The `static_rnn()` function creates an unrolled RNN network by chaining cells. The following
code creates the exact same model as the previous one.

In [10]:
tf.reset_default_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
Y0, Y1 = output_seqs

init = tf.global_variables_initializer()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


**(?)** From **manually** doing RNN to **`static_rnn()`**, what do we gain?<br>
**(R)**<br>
- We don't have to create `tf.Variable`'s by ourselves anymore, saving some work like creating them, initializing them, etc.

In [11]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [12]:
print(f"Y0_val =\n{Y0_val}")
print()
print(f"Y1_val =\n{Y1_val}")

Y0_val =
[[-0.93096346 -0.7259365  -0.7300388   0.22798158  0.38557476]
 [-0.998821   -0.98619145 -0.99990207 -0.6016948  -0.9113301 ]
 [-0.99998057 -0.99939144 -1.         -0.9251532  -0.99809295]
 [ 0.9989888  -0.9424999  -0.9999802  -0.9504026  -0.99992627]]

Y1_val =
[[-0.9995659  -0.99981296 -1.         -0.99927443 -0.99998605]
 [ 0.5346995  -0.5864016  -0.5589596  -0.93255424  0.44668883]
 [-0.92909604 -0.9942717  -0.99999976 -0.9993018  -0.99867207]
 [-0.5831156  -0.18158859 -0.99908304 -0.95810765 -0.96227044]]


What was shown next on the book was the use of
- `tf.unstack()`
  - `tf.stack()`
- `tf.transpose()`
to allow input of a different (but equivalent) shape.

Note that the code and shape of tensors in `BasicRNNCell()` and in `static_rnn()` remains
identical to previously. We just tried to modify the model's input and output shape.

In [13]:
n_steps = 2
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
# At this stage, shape(output_seqs) = (n_steps, None, n_inputs) (or at least approximately)
# But we'd like to have sth of shape (None, n_steps, n_inputs)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [14]:
init = tf.global_variables_initializer()
X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7]],  # instance 0
    [[3,4,5], [0,0,0]],  # instance 1
    [[6,7,8], [6,5,4]],  # instance 2
    [[9,0,1], [3,2,1]],  # instance 3
])

In [15]:
sess = tf.InteractiveSession()
init.run()
outputs_val = outputs.eval(feed_dict={X: X_batch})
sess.close()
outputs_val

array([[[-0.73817766,  0.20324594,  0.88626933,  0.6419878 ,
         -0.902788  ],
        [-0.99449295,  0.9989233 ,  1.        ,  0.9540346 ,
         -1.        ]],

       [[-0.97996455,  0.9323643 ,  0.99990714,  0.8636019 ,
         -0.9999784 ],
        [ 0.13735573, -0.7017959 ,  0.4060293 ,  0.888072  ,
          0.38207084]],

       [[-0.99864125,  0.99630654,  0.9999999 ,  0.95204085,
         -1.        ],
        [-0.9315469 ,  0.952682  ,  0.9999921 ,  0.9472138 ,
         -0.9999964 ]],

       [[-0.24839847,  0.9999641 ,  0.97042406,  0.8664661 ,
         -0.9990241 ],
        [-0.06951729,  0.596474  ,  0.99385506,  0.8481856 ,
         -0.9929457 ]]], dtype=float32)

### OOM (Out-of-memory) error
**(?)** I don't quite understand the oom here. Does it mean that `dynamic_rnn()` won't suffer from oom?

**(R)** `dynamic_rnn()`'s main purpose is not to prevent from OOM errors, although it is capable of that. By using `dynamic_rnn()`,
- we can have the convenient input and output shapes as above **without** having to `unstack()`, `transpose()`, etc.
- we can set `swap_memory=True` to swap GPU's memory to CPU's memory to prevent OOM errors
- we can handle variable length input sequences, i.e. `n_steps` may be different for each instance

In [16]:
n_inputs = 3 
n_steps = 2
n_neurons = 5
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
#X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
outputs, states

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


(<tf.Tensor 'rnn/transpose_1:0' shape=(?, 2, 5) dtype=float32>,
 <tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 5) dtype=float32>)

In [17]:
init = tf.global_variables_initializer()
X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7]],  # instance 0
    [[3,4,5], [0,0,0]],  # instance 1
    [[6,7,8], [6,5,4]],  # instance 2
    [[9,0,1], [3,2,1]],  # instance 3
])

sess = tf.InteractiveSession()
init.run()
outputs_val = outputs.eval(feed_dict={X: X_batch})
sess.close()
outputs_val

array([[[ 0.61623365,  0.82570535,  0.40422666,  0.5780889 ,
          0.45100054],
        [-0.98775417,  0.98486006,  0.36937082, -0.876407  ,
          1.        ]],

       [[ 0.07657859,  0.98531246,  0.5585418 ,  0.56603247,
          0.99931467],
        [-0.14702493, -0.68977165,  0.5135126 , -0.8336407 ,
         -0.5706824 ]],

       [[-0.51201165,  0.9988539 ,  0.68194014,  0.5537288 ,
          0.9999996 ],
        [-0.9840562 ,  0.27693594,  0.63996565, -0.9733648 ,
          0.9999914 ]],

       [[-0.99996537, -0.9982177 , -0.9996932 ,  0.28515962,
          0.99995023],
        [-0.97963065, -0.67000806, -0.34270397, -0.5824971 ,
          0.9984212 ]]], dtype=float32)

#### Handling variable length input sequences

In [18]:
n_inputs = 3 
n_steps = 3
n_neurons = 5
tf.reset_default_graph()
seq_length = tf.placeholder(tf.int32, [None])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
#X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                    sequence_length=seq_length)
outputs, states

Instructions for updating:
Use tf.cast instead.


(<tf.Tensor 'rnn/transpose_1:0' shape=(?, 3, 5) dtype=float32>,
 <tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 5) dtype=float32>)

In [19]:
init = tf.global_variables_initializer()
X_batch = np.array([
    # t = 0    t = 1
    [[0,1,2], [9,8,7], [0,0,0]],  # instance 0 (length 2, padded with 0's)
    [[3,4,5], [0,0,0], [0,0,0]],  # instance 1 (length 1)
    [[6,7,8], [6,5,4], [0,0,0]],  # instance 2 (length 3, the last happens to be 0's)
    [[9,0,1], [3,2,1], [4,5,6]],  # instance 3 (length 3)
])
seq_length_batch = [2,1,3,3]

In [20]:
sess = tf.InteractiveSession()
init.run()
outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})
sess.close()
outputs_val

array([[[ 0.45970207,  0.5309875 ,  0.25191087, -0.29899323,
          0.6064514 ],
        [ 0.81458396,  0.95972246, -0.7108613 , -0.9919086 ,
          0.7231157 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.82482606,  0.90868115, -0.06679729, -0.8551859 ,
          0.86663973],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.95141786,  0.9851668 , -0.37243372, -0.97767013,
          0.9591799 ],
        [ 0.25544474,  0.839287  , -0.12022333, -0.9667628 ,
          0.23046581],
        [-0.16154294, -0.49780732,  0.5405249 , -0.26665965,
         -0.12828113]],

       [[-0.99302036,  0.99774855, -0.99991006, -0.9985037 ,
          0.49413767],
        [-0.04516888, -0.48170552, -0.02056463, -0.522182  ,
         -0.20978336],
        [ 0.8796154 ,  0.958476  , -0.21074432, -0.9573855 ,
         

In [21]:
states_val

array([[ 0.81458396,  0.95972246, -0.7108613 , -0.9919086 ,  0.7231157 ],
       [ 0.82482606,  0.90868115, -0.06679729, -0.8551859 ,  0.86663973],
       [-0.16154294, -0.49780732,  0.5405249 , -0.26665965, -0.12828113],
       [ 0.8796154 ,  0.958476  , -0.21074432, -0.9573855 ,  0.9144974 ]],
      dtype=float32)

Note that `states_val` records the last non-zero output from each instances.